set up env variables (e.g. in a .env file if using vscode)

- "KEY", for your openai API key
- "PROMPT_SYSTEM" (optionally) if you want to give your chatbot a default personality (e.g. helpful expert in area x)

In [ ]:
#dependencies

#%pip install openai
#%pip install ipywidgets
#%pip install pyperclip

### API

In [ ]:

import os
import openai
import ipywidgets as widgets
from IPython.display import display
import pyperclip

#api key
openai.api_key = os.getenv("KEY")
#default personality / behavior
default_system = os.getenv("PROMPT_SYSTEM")

#chat model parameters (ChatGPT)
chat_model = "gpt-3.5-turbo"
chat_temp = 0.7
chat_convo_limit = 3000
chat_token_limit = 1000

if (default_system is not None):
    #initializing chat history with the system prompt
    conversation = [{"role": "system", "content": default_system}]
else:
    conversation = []
    
#the maximum context window (short term memory) is 4096 tokens, so we need to keep track of token usage to limit the risk running into an API error; the webUI is using a sliding window to manage around this limit
token_count = 0


#API calls to OpenAI
def get_gpt (prompt):
    global token_count
    global conversation

    #if our current token_count is above limit then we remove the oldest message from the conversation, if there is a personality set up in default_system then the 2nd oldest
    if token_count > chat_convo_limit:
   
        prompt_length = len(prompt)

        while prompt_length > 0 :
   
            if (default_system is not None):
                item_length = len(conversation[1]['content'])
                conversation.pop(1)
            else:
                item_length = len(conversation[0]['content'])
                conversation.pop(0)

            prompt_length = prompt_length - item_length
        

    conversation.append({"role": "user", "content": prompt})
    response = openai.ChatCompletion.create(
            #gpt-3.5-turbo is the current equivalent of what's behind the web UI
            model=chat_model, 
            messages=conversation,
            #the higher the temperature the more creative / diverse the responses will be, minimum is 0 (deterministic), maximum is currently 2 (too much)
            temperature = chat_temp,
            max_tokens = chat_token_limit,
            n = 1            
        )
    message = response.choices[0].message.content
    token_count = response.usage.total_tokens

    conversation.append({"role": "assistant", "content": message})
    return response



### UI

In [ ]:
from IPython.display import HTML
import ipywidgets as widgets

#HTML formatting for the output window
html_template = """
<style>
div {{white-space: pre-wrap;font-size: 18px;}}
</style>
<div>
<div>
{}
</div>
<div>
{}
</div>
</div>
"""

#displaying messages
def render_message(message):
    if message["role"] == "assistant":
        return(f"<i>ChatGPT</i>: {message['content']}")
    else:
        return(f"<b>You</b>: {message['content']}")
    

# handler for copy button's on_click event
def handle_copy(button):
    global conversation
    
    if (default_system is not None):
        conversation_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation[1:]])
    else:
        conversation_str = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])

    # Add conversation string to clipboard
    pyperclip.copy(conversation_str)


# handler for send button's on_click event
def handle_input(button):
    user_input = input_widget.value
    output.append_display_data("--------------message sent--------------")
    input_widget.value = ''
    
    response = get_gpt(user_input)
    latest_messages = html_template.format(render_message(conversation[-2]),render_message(conversation[-1]))
    output.append_display_data(HTML(latest_messages))

    print('current token count: ', token_count)


output = widgets.Output(layout={'border': '1px solid black','width': '100%'},contextmenu=True)

#input window
input_widget = widgets.Textarea(
    value='',
    placeholder='Type your message here...',
    description='Input:',
    disabled=False,
    layout={'width': '100%', 'height': 'auto'},
    rows = 1
)

#resizing input window as the input gets longer    
def get_bigger(args):        
    input_widget.rows = input_widget.value.count('\n') + 1

input_widget.observe(get_bigger,'value')

#the send button
send_button = widgets.Button(description='Send', layout={'width': '100%'})
send_button.on_click(handle_input)

#the copy to clipbord button
copy_button = widgets.Button(description='Copy Chat to Clipboard', layout={'width': '100%'})
copy_button.on_click(handle_copy)

# Display the UI
display(widgets.VBox([copy_button,output,input_widget,send_button]))



In [ ]:
conversation